## MLOps Tools: MLflow and Hugging Face

> https://www.coursera.org/learn/mlops-mlflow-huggingface-duke/home/week/1

---

### Introduction

In [3]:
from random import choice
import mlflow
from mlflow import log_metric, log_param, log_artifact

In [6]:
log_param("threshold", 3)
log_param("verbosity", "DEBUG")

'DEBUG'

In [3]:
log_metric("timestamp", 3000)
log_metric("TTC", 33)

In [5]:
log_artifact('./data/test.txt')

For the next task we would like to track some random metrics

Let us first  create an experiment:

```
> mlflow experiments create --experiment-name log-metrics-demo
> Created experiment 'log-metrics-demo' with id 384253852139107280
```



In [7]:
mlflow.set_experiment("log-metrics-demo")

<Experiment: artifact_location='file:///Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn%20MLflow/MLFLow%20Course%20-%20Duke%20University/mlruns/384253852139107280', creation_time=1690609937121, experiment_id='384253852139107280', last_update_time=1690609937121, lifecycle_stage='active', name='log-metrics-demo', tags={}>

In [9]:
# lets log some metrics
metric_names = ["cpu", "ram", "disk"]
percentages = [i for i in range(0, 101)]

for i in range(10):
    print (choice(metric_names), choice(percentages))
    log_metric(key=choice(metric_names), value=choice(percentages))

disk 87
disk 78
disk 34
ram 77
disk 82
disk 19
ram 51
cpu 47
disk 47
cpu 47


Basically what happened here is
- we created a new experiment
- we set this experiment in mlflow
- we logged a bunch of metrics under this experiment

So if you see in /mlruns, there should be a folder corresponding to the id of the experiment we just created

![](https://imgur.com/En296nb.png)

In [17]:
# if an active run exists, end it
if mlflow.active_run():
    mlflow.end_run()

In [18]:
mlflow.set_experiment("log-metrics-demo")

<Experiment: artifact_location='file:///Users/shaunaksen/Documents/personal-projects/ML_Deployment/Learn%20MLflow/MLFLow%20Course%20-%20Duke%20University/mlruns/384253852139107280', creation_time=1690609937121, experiment_id='384253852139107280', last_update_time=1690609937121, lifecycle_stage='active', name='log-metrics-demo', tags={}>

In [19]:
# lets log some metrics
metric_names = ["cpu"]
percentages = [i for i in range(0, 101)]

with mlflow.start_run(run_name="cpu_new"):

    for i in range(400):
        # print (choice(metric_names), choice(percentages))
        log_metric(key="cpu_new", value=choice(percentages))

mlflow.end_run()

We can also query the metrics using the mlflow UI search box:

![](https://imgur.com/GszAvYj.png)